In [33]:
import pyreadr
from conllu import parse
import pandas as pd
import numpy as np
from io import open
from conllu import parse_incr


In [34]:
data_file_dev = open("it_partut-ud-dev.conllu", "r", encoding="utf-8")
data_file_train = open("it_partut-ud-train.conllu", "r", encoding="utf-8")
data_file_test = open("it_partut-ud-test.conllu", "r", encoding="utf-8")
sentences = []
words_lemmas = []

for tokenlist in parse_incr(data_file_dev):
    w_l = []
    for token in tokenlist:
        w_l = [token['form'], token['upos'], token['lemma']]
        words_lemmas.append(w_l)
        w_l = []
    sentences.append(tokenlist.metadata['text'])
    

for tokenlist in parse_incr(data_file_train):
    w_l = []
    for token in tokenlist:
        w_l = [token['form'], token['upos'], token['lemma']]
        words_lemmas.append(w_l)
        w_l = []
    sentences.append(tokenlist.metadata['text'])


    
for tokenlist in parse_incr(data_file_test):
    w_l = []
    for token in tokenlist:
        w_l = [token['form'],token['upos'],token['lemma']]
        words_lemmas.append(w_l)
        w_l = []
    sentences.append(tokenlist.metadata['text'])
    
# all lower words_lemmas
words_lemmas = [[x.lower() for x in y] for y in words_lemmas]

df = pd.DataFrame(words_lemmas, columns = ['word', 'tag', 'lem'])

In [35]:
dev = pd.read_csv('./dev.csv', sep='\t', header=0)
test = pd.read_csv('./test.csv', sep='\t', header=0)

dev['word'] = dev['word'].str.lower()
dev['tag'] = dev['tag'].str.lower()
dev['lem'] = dev['lem'].str.lower()

test['word'] = test['word'].str.lower()
test['tag'] = test['tag'].str.lower()
test['lem'] = test['lem'].str.lower()

dev = dev.dropna()
test = test.dropna()

print(dev)
        

            word       tag       lem
0             mi  pron_per        mi
1      riferisco    v_gvrb  riferire
2             al    prep_a        al
3         lavoro        nn    lavoro
4           dove    conj_s      dove
...          ...       ...       ...
17308   &egrave;  v_essere    essere
17309         la       art        la
17310        mia  pron_pos       mia
17311          '     p_apo         '
17312          .     p_eos         .

[17301 rows x 3 columns]


In [36]:
# list all possible pos-tags
pos_tags = []
for i in words_lemmas:
    pos_tags.append(i[2])
    
pos_tags = list(set(pos_tags))

dev_pos = dev['tag'].tolist()
test_pos = test['tag'].tolist()

# create dict where the key is old pos-tag and the value is new pos-tag
dict_pos = {}
for i in range(len(dev_pos)):
    if 'v_' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'verb'
    elif 'p_' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'punct'
    elif dev_pos[i] == 'nn':
        dict_pos[dev_pos[i]] = 'noun'
    elif 'adj' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'adj'
    elif 'nn_p' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'propn'
    elif 'prep_a' in dev_pos[i]:
        dict_pos[dev_pos[i]] = '_'
    elif 'conj_s' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'sconj'
    elif 'adv' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'adv'
    elif 'conj_c' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'cconj'
    elif 'pron_rel' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'pron'
    elif  dev_pos[i] == 'prep':
        dict_pos[dev_pos[i]] = 'adp'
    elif 'c_num' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'num'
    elif 'art' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'det'
    elif 'pron_ind' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'pron'
    elif 'pron_per' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'pron'
    elif 'pron_dim' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'pron'
    elif 'pron_ies' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'pron'
    elif 'pron_pos' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'pron'
    elif 'int' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'X'
    elif 'p_apo' in dev_pos[i]:
        dict_pos[dev_pos[i]] = 'punct'


In [37]:
# replacing old pos-tags with new pos-tags
for index, row in dev.iterrows():
    row['tag'] = dict_pos[row['tag']]
    
for index, row in test.iterrows():
    row['tag'] = dict_pos[row['tag']]


In [ ]:
#write on csv
test.to_csv('./test_new.csv', sep="\t",escapechar='\t', index=False)

In [ ]:
# concatenate
new = pd.concat([dev, df])

# delete where tag is x
new = new[new.tag != 'X']

In [ ]:
# write on csv
new.to_csv('./dev_new.csv', sep="\t",escapechar='\t', index=False)